In [2]:
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## 一.瓜子车源数据匹配

In [8]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# reload(sys)
# sys.setdefaultencoding('utf8')

ENCODING = 'utf-8'
# PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
# PRODUCE_DB_USER = 'pingjia'
# PRODUCE_DB_PASSWD = 'De32wsxC'

PRODUCE_DB_ADDR_OUTTER = '101.201.143.74'
PRODUCE_DB_USER = 'leidengjun'
PRODUCE_DB_PASSWD = 'ldj_DEV_~!'

PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'
PRODUCE_CHINA_USED_CAR_ESTIMATE_ENGINE = 'mysql+pymysql://' + PRODUCE_DB_USER + ':' + PRODUCE_DB_PASSWD + '@' + PRODUCE_DB_ADDR_OUTTER + '/china_used_car_estimate?charset=utf8'
engine = create_engine(PRODUCE_CHINA_USED_CAR_ESTIMATE_ENGINE)

yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
start = str(yesterday.year) + '-' + str(yesterday.month) + '-' + str(yesterday.day) + ' 00:00:00'
today = datetime.datetime.now()
end = str(today.year) + '-' + str(today.month) + '-' + str(today.day) + ' 00:00:00'

# query_sql = 'select * from crawler_car_source ' \
#             'where create_time >= \''+start+'\' and  create_time < \''+end+'\' '
query_sql = 'select source_id,detail_name,year,month,mile,price,city,domain,transfer_owner from crawler_car_source'
origin_train = pd.read_sql_query(query_sql, engine)
origin_train.to_csv('./origin_train.csv', index=False)
print('finish query, num:'+str(len(origin_train)))

finish query, num:176466


### 一.车源更新脚本

In [92]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

reload(sys)
sys.setdefaultencoding('utf8')

def process_price_bn(df):
    if df['price_bn'] == '暂无报价':
        return np.NAN
    temp = re.findall(r'万',df['price_bn'])
    if len(temp) > 1:
        text = df['price_bn'].split('~')[1]
    else:
        text = df['price_bn']
    text = text.replace('万','')
    return float(text)

def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """    
    text = df[column_name]
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('-', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

def process_url(df):
    return df['url'].replace('https://','')

def process_eval_pirce(df):
    random_price = random.randint(int(-df['price']*0.95*0.03*10000),int(df['price']*0.95*0.03*10000))
    if (df['price'] * 0.95 + random_price/10000) < 0.5:
        return df['price']
    return float('%.2f' % (df['price'] * 0.95 + random_price/10000))

def process_gpj_index(df):
    k = 2
    b = abs(df['price'] * 0.95 - df['eval_price']) / df['eval_price']
    return pd.Series([float('%.2f' % (10 * (math.e ** (-k*b)))),int((10 * (math.e ** (-k*b)))*100000000)])

def process_thumbnail(df):
    url = []
    thumbnail = json.loads(df['thumbnail'])
    if len(thumbnail) < 10:
        return pd.Series([np.NaN,np.NaN])
    url.append(thumbnail[0]['images'][0]['image'])
    url.append(thumbnail[1]['images'][0]['image'])
    url.append(thumbnail[5]['images'][0]['image'])
    url.append(thumbnail[6]['images'][0]['image'])
    url.append(thumbnail[7]['images'][0]['image'])
    url.append(thumbnail[8]['images'][0]['image'])
    url.append(thumbnail[9]['images'][0]['image'])
    url.append(thumbnail[10]['images'][0]['image'])
    return pd.Series([thumbnail[0]['images'][0]['image'],url])

#########################################step1
car_autohome_all = pd.read_csv('./car_autohome_all.csv')
car_autohome_all = car_autohome_all.loc[:,['detail_slug','volume','control']]

pcm = pd.read_csv('./province_city_map.csv')
pcm = pcm.loc[:,['city','province']]

gpj_model = pd.read_csv('./combine_model.csv',low_memory=False)
gpj_model = gpj_model.loc[:,['slug','parent','classified_slug','popular']]
gpj_model = gpj_model.rename(columns={'slug':'model_slug','parent':'brand_slug','classified_slug':'body_style'})

gpj_detail = pd.read_csv('./combine_detail.csv',low_memory=False)
gpj_detail = gpj_detail.loc[:,['global_slug','volume','control','car_autohome_detail_id','detail_model_slug']]
gpj_detail = gpj_detail.rename(columns={'global_slug':'model_slug','car_autohome_detail_id':'detail_slug'})
gpj_detail = gpj_detail.merge(gpj_model,how='left',on=['model_slug'])
gpj_detail = gpj_detail.drop_duplicates(['detail_slug','volume','control']).reset_index(drop=True)

train_final = pd.read_csv('./train_final.csv')
train_final = train_final.rename(columns={'final_text':'title'})

##########################################step2
origin_train = pd.read_csv('./origin_train.csv')
origin_train = origin_train.drop(['brand_name','model_name'],axis=1)
origin_train = origin_train.loc[(origin_train['source_id'].notnull())&(origin_train['detail_name'].notnull())&(origin_train['price'] > 0), :].reset_index(drop=True)
origin_train['title'] = origin_train.apply(delete_str_useless, args=('detail_name',), axis=1)
origin_train = origin_train.rename(columns={'detail_name':'origin_name'})

data = origin_train.copy()
print('cars count:', len(data))
final = data.merge(train_final, how='left', on=['title'])
final = final.sort_values(by=['title'])
match = final.loc[(final['online_year'].notnull())&(final['city'].notnull()), :].reset_index(drop=True)
print('have match count:',len(match), 'rate:', len(match)/len(data))

cur_time = datetime.datetime.now()
car_source = match.drop(['id','car_application','labels','title','brand_slug','model_slug','online_year','brand_name','model_name','detail_name'],axis=1)
car_source['pub_time'] = car_source['create_time']
car_source['status'] = 'sale'
car_source['source_type'] = 'cpersonal'
car_source['process_status'] = 'S'
car_source['global_sibling'] = 0
car_source['city_sibling'] = 0
car_source['dealer_sibling'] = 0
car_source['domain_sibling'] = 0
car_source['expired_at'] = datetime.datetime.now() + datetime.timedelta(days=10)
car_source['domain_sibling'] = 0
car_source['created_on'] = cur_time 
car_source['cs_clean'] = 0
car_source = car_source.merge(pcm, how='left', on=['city'])
car_source = car_source.merge(car_autohome_all, how='left', on=['detail_slug'])
car_source = car_source.merge(gpj_detail,how='left',on=['detail_slug','volume','control'])
car_source = car_source.rename(columns={'origin_name':'title','detail_model_slug':'model_detail_slug','popular':'popularity'})
car_source = car_source.drop(['create_time','detail_slug'],axis=1)
car_source = car_source.loc[(car_source['model_detail_slug'].notnull()),:].reset_index(drop=True)
car_source = car_source.drop_duplicates(['url'])
car_source['url'] = car_source.apply(process_url, axis=1)
car_source['eval_price'] = car_source.apply(process_eval_pirce, axis=1)
car_source[['gpj_index','rank']] = car_source.apply(process_gpj_index, axis=1)
car_source[['thumbnail','car_image']] = car_source.apply(process_thumbnail, axis=1)
car_source = car_source.loc[(car_source['car_image'].notnull()),:].reset_index(drop=True)
car_source = car_source.sort_values(by=['url','pub_time'],ascending=[True,False]).reset_index(drop=True)
car_source = car_source.drop_duplicates(['url']).reset_index(drop=True)

        
ENCODING = 'utf-8'
PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
PRODUCE_DB_USER = 'pingjia'
PRODUCE_DB_PASSWD = 'De32wsxC'
PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'

engine = create_engine(PRODUCE_PINGJIA_ENGINE, encoding=ENCODING)  

with engine.begin() as con:
    insert_car_source = car_source.drop(['source_id','car_image','description','conditions','contact','region','company_name',
                'maintenance','insurance_money','quality_assurance','transfer_owner','annual_insurance',
                'compulsory_insurance','business_insurance','price_bn'],axis=1)
#     columns = str(list(insert_car_source.columns))
#     columns = columns[1:len(columns) - 1]
#     columns = columns.replace('\'', '')
    for i in range(0, len(insert_car_source)):
        url = insert_car_source.loc[i, 'url']
        source = insert_car_source.loc[i:i,:]
#         value = str(list(insert_car_source.loc[i, :].values))
#         value = value[1:len(value) - 1]
#         value = re.sub(r' nan', ' null', value)
#         sql = 'INSERT INTO car_source(' + columns + ') VALUES (' + value + ') ON DUPLICATE KEY UPDATE url=\'' +url+ '\' '
#         con.execute(sql)
        sql = 'delete from car_source where url=\'' +url+ '\' '
        con.execute(sql)
        con.execute('commit')
        source.to_sql(name='car_source', if_exists='append', con=engine, index=False)
        
        sql = 'select id,thumbnail from car_source where url=\'' +url+ '\' '
        query_car_source = pd.read_sql_query(sql, engine)
        final_car_source = car_source.loc[i:i,:].merge(query_car_source,how='left',on=['thumbnail'])
        car_image = pd.DataFrame()
        car_image['image'] = pd.Series(final_car_source.loc[0,'car_image'])
        car_id = final_car_source.loc[0,'id']
        car_image['car_id'] = car_id
        car_image['create_time'] = cur_time
        sql = 'delete from car_image where car_id = ' + str(car_id)
        con.execute(sql)
        con.execute('commit')
        car_image.to_sql(name='car_image', if_exists='append', con=engine, index=False)
        
        car_detail_info = final_car_source.copy()
        car_detail_info = car_detail_info.loc[:,['conditions','description','contact','region','company_name','annual_insurance','compulsory_insurance',
                                                'business_insurance','transfer_owner','maintenance','insurance_money','quality_assurance','id','price_bn']]
        car_detail_info['update_time'] = cur_time.strftime('%Y-%m-%d %H:%M:%S')
        car_detail_info['condition_score'] = ''
        car_detail_info = car_detail_info.rename(columns={'id':'car_id','annual_insurance':'examine_insurance','compulsory_insurance':'mandatory_insurance',
                                                         'conditions':'condition'})
        car_detail_info['mdn_status'] = 'P'
        car_detail_info['dealer_status'] = 0
        if car_detail_info.loc[0,'examine_insurance'] == '已过期':
            car_detail_info['examine_insurance'] = datetime.datetime.now()
        else:
            temp = car_detail_info.loc[0,'examine_insurance']
            temp = temp.replace('.-','.')
            car_detail_info['examine_insurance'] = pd.to_datetime(temp,format="%Y-%m")
        
        if car_detail_info.loc[0,'mandatory_insurance'] == '已过期':
            car_detail_info['mandatory_insurance'] = datetime.datetime.now()
        else:
            temp = car_detail_info.loc[0,'mandatory_insurance']
            temp = temp.replace('.-','.')
            car_detail_info['mandatory_insurance'] = pd.to_datetime(temp,format="%Y-%m")
        
        if car_detail_info.loc[0,'business_insurance'] == '已过期':
            car_detail_info['business_insurance'] = datetime.datetime.now()
        else:
            temp = car_detail_info.loc[0,'business_insurance']
            temp = temp.replace('.-','.')
            car_detail_info['business_insurance'] = pd.to_datetime(temp,format="%Y-%m")

        sql = 'delete from car_detail_info where car_id = ' + str(car_id)
        con.execute(sql)
        con.execute('commit')
        car_detail_info.to_sql(name='car_detail_info', if_exists='append', con=engine, index=False)
        print i
con.close()

cars count: 10
have match count: 10 rate: 1.0


### 二.估值数据库更新脚本

In [2]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

reload(sys)
sys.setdefaultencoding('utf8')

ENCODING = 'utf-8'
PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
PRODUCE_DB_USER = 'pingjia'
PRODUCE_DB_PASSWD = 'De32wsxC'
PRODUCE_USED_CAR_ESTIMATE_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/china_used_car_estimate?charset=utf8'
engine = create_engine(PRODUCE_USED_CAR_ESTIMATE_ENGINE, encoding=ENCODING)  

global_model_mean = pd.read_csv('./global_model_mean.csv')
global_model_mean = global_model_mean.drop(['listed_year'], axis=1)

with engine.begin() as con:
    sql = 'delete from valuate_global_model_mean where update_time < \''+  datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\' ' 
    con.execute(sql)
    con.execute('commit')
con.close()

global_model_mean.to_sql(name='valuate_global_model_mean', if_exists='append', con=engine, index=False)

'2019.1'

In [19]:
def process_price_bn(df):
    if df['price_bn'] == '暂无报价':
        return np.NAN
    temp = re.findall(r'万',df['price_bn'])
    if len(temp) > 1:
        text = df['price_bn'].split('~')[1]
    else:
        text = df['price_bn']
    text = text.replace('万','')
    return float(text)

def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """    
    text = df[column_name]
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('-', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

# 判断是否有重复final_text
origin_train = pd.read_csv('./origin_train.csv')

origin_train = origin_train.loc[(origin_train['source_id'].notnull())&(origin_train['detail_name'].notnull())&(origin_train['price'] > 0)&(origin_train['transfer_owner'] == 0), :].reset_index(drop=True)
origin_train['title'] = origin_train.apply(delete_str_useless, args=('detail_name',), axis=1)
origin_train = origin_train.rename(columns={'detail_name':'origin_name'})
train_final = pd.read_csv('../tmp/train/train_final.csv')
train_final = train_final.rename(columns={'final_text':'title'})

In [21]:
# data = wait_predict.loc[(wait_predict['domain'].isin(['guazi.com'])), :]
data = origin_train.copy()
print('瓜子车源数据量:', len(data))
final = data.merge(train_final, how='left', on=['title'])
final = final.sort_values(by=['title'])
match = final.loc[(final['online_year'].notnull())&(final['city'].notnull()), :].reset_index(drop=True)
print('瓜子车源数据匹配量:',len(match), '匹配率:', len(match)/len(data))
match.info()
match.to_csv('/home/ml/ProgramProject/evaluation-predict/tmp/train/car_source_match.csv', index=False)

瓜子车源数据量: 130718
瓜子车源数据匹配量: 122346 匹配率: 0.935953732462247
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122346 entries, 0 to 122345
Data columns (total 18 columns):
source_id         122346 non-null int64
origin_name       122346 non-null object
year              122346 non-null int64
month             122346 non-null float64
mile              122346 non-null float64
price             122346 non-null float64
city              122346 non-null object
domain            122346 non-null object
transfer_owner    122346 non-null float64
title             122346 non-null object
brand_slug        122346 non-null float64
model_slug        122346 non-null float64
detail_slug       122346 non-null float64
online_year       122346 non-null float64
price_bn          122346 non-null float64
brand_name        122346 non-null object
model_name        122346 non-null object
detail_name       122346 non-null object
dtypes: float64(9), int64(2), object(7)
memory usage: 16.8+ MB


In [26]:
match.loc[(match['detail_slug'] == 12128),:]

,source_id,origin_name,year,month,mile,price,city,domain,transfer_owner,title,brand_slug,model_slug,detail_slug,online_year,price_bn,brand_name,model_name,detail_name
95086,3007989126,福特 福克斯 2012款 两厢 1.6L 手动风尚型,2013,3.0,8.0,6.00,珠海,guazi.com,0.0,福特福克斯2012款两厢1.6l手动风尚型,8.0,364.0,12128.0,2012.0,13.29,福特,福克斯,2012款 两厢 1.6L 手动风尚型
95087,3008165809,福特 福克斯 2012款 两厢 1.6L 手动风尚型,2015,2.0,6.9,5.90,重庆,guazi.com,0.0,福特福克斯2012款两厢1.6l手动风尚型,8.0,364.0,12128.0,2012.0,13.29,福特,福克斯,2012款 两厢 1.6L 手动风尚型
95088,3007571300,福特 福克斯 2012款 两厢 1.6L 手动风尚型,2015,8.0,6.3,6.00,保定,guazi.com,0.0,福特福克斯2012款两厢1.6l手动风尚型,8.0,364.0,12128.0,2012.0,13.29,福特,福克斯,2012款 两厢 1.6L 手动风尚型
95089,3007051871,福特 福克斯 2012款 两厢 1.6L 手动风尚型,2014,10.0,7.2,5.89,柳州,guazi.com,0.0,福特福克斯2012款两厢1.6l手动风尚型,8.0,364.0,12128.0,2012.0,13.29,福特,福克斯,2012款 两厢 1.6L 手动风尚型
95090,3008216884,福特 福克斯 2012款 两厢 1.6L 手动风尚型,2014,12.0,4.6,6.00,泰州,guazi.com,0.0,福特福克斯2012款两厢1.6l手动风尚型,8.0,364.0,12128.0,2012.0,13.29,福特,福克斯,2012款 两厢 1.6L 手动风尚型
95091,3008244803,福特 福克斯 2012款 两厢 1.6L 手动风尚型,2013,1.0,6.9,5.23,昆明,guazi.com,0.0,福特福克斯2012款两厢1.6l手动风尚型,8.0,364.0,12128.0,2012.0,13.29,福特,福克斯,2012款 两厢 1.6L 手动风尚型
95092,3008171713,福特 福克斯 2012款 两厢 1.6L 手动风尚型,2014,9.0,12.9,6.20,大连,guazi.com,0.0,福特福克斯2012款两厢1.6l手动风尚型,8.0,364.0,12128.0,2012.0,13.29,福特,福克斯,2012款 两厢 1.6L 手动风尚型
95093,3007917622,福特 福克斯 2012款 两厢 1.6L 手动风尚型,2014,7.0,4.2,6.30,自贡,guazi.com,0.0,福特福克斯2012款两厢1.6l手动风尚型,8.0,364.0,12128.0,2012.0,13.29,福特,福克斯,2012款 两厢 1.6L 手动风尚型
95094,3007379869,福特 福克斯 2012款 两厢 1.6L 手动风尚型,2014,2.0,4.8,6.20,绵阳,guazi.com,0.0,福特福克斯2012款两厢1.6l手动风尚型,8.0,364.0,12128.0,2012.0,13.29,福特,福克斯,2012款 两厢 1.6L 手动风尚型
95095,3008024265,福特 福克斯 2012款 两厢 1.6L 手动风尚型,2012,12.0,3.0,6.65,济南,guazi.com,0.0,福特福克斯2012款两厢1.6l手动风尚型,8.0,364.0,12128.0,2012.0,13.29,福特,福克斯,2012款 两厢 1.6L 手动风尚型


In [23]:
np.median(list(match.loc[(match['detail_slug'] == 18211),'price']))

8.0

### 二.入库car_source

In [73]:
def process_url(df):
    return df['url'].replace('https://','')

def process_eval_pirce(df):
    random_price = random.randint(int(-df['price']*0.95*0.03*10000),int(df['price']*0.95*0.03*10000))
    return float('%.2f' % (df['price'] * 0.95 + random_price/10000))

def process_gpj_index(df):
    k = 2
    b = abs(df['price'] * 0.95 - df['eval_price']) / df['eval_price']
    return pd.Series([float('%.2f' % (10 * (math.e ** (-k*b)))),int((10 * (math.e ** (-k*b)))*100000000)])

def process_thumbnail(df):
    url = []
    thumbnail = json.loads(df['thumbnail'])
    if len(thumbnail) < 10:
        return pd.Series([np.NaN,np.NaN])
    url.append(thumbnail[0]['images'][0]['image'])
    url.append(thumbnail[1]['images'][0]['image'])
    url.append(thumbnail[5]['images'][0]['image'])
    url.append(thumbnail[6]['images'][0]['image'])
    url.append(thumbnail[7]['images'][0]['image'])
    url.append(thumbnail[8]['images'][0]['image'])
    url.append(thumbnail[9]['images'][0]['image'])
    url.append(thumbnail[10]['images'][0]['image'])
    return pd.Series([thumbnail[0]['images'][0]['image'],url])
    
car_autohome_all = pd.read_csv('../tmp/train/car_autohome_all.csv')
car_autohome_all = car_autohome_all.loc[:,['detail_slug','volume','control']]

pcm = pd.read_csv('../tmp/train/province_city_map.csv')
pcm = pcm.loc[:,['city','province']]

gpj_model = pd.read_csv('../tmp/train/combine_model.csv',low_memory=False)
gpj_model = gpj_model.loc[:,['slug','parent','classified_slug','popular']]
gpj_model = gpj_model.rename(columns={'slug':'model_slug','parent':'brand_slug','classified_slug':'body_style'})

gpj_detail = pd.read_csv('../tmp/train/combine_detail.csv',low_memory=False)
gpj_detail = gpj_detail.loc[:,['global_slug','volume','control','car_autohome_detail_id','detail_model_slug']]
gpj_detail = gpj_detail.rename(columns={'global_slug':'model_slug','car_autohome_detail_id':'detail_slug'})
gpj_detail = gpj_detail.merge(gpj_model,how='left',on=['model_slug'])
gpj_detail = gpj_detail.drop_duplicates(['detail_slug','volume','control']).reset_index(drop=True)

cur_time = datetime.datetime.now()

car_source = match.drop(['id','car_application','labels','title','brand_slug','model_slug','online_year','brand_name','model_name','detail_name'],axis=1)
car_source['pub_time'] = car_source['create_time']
car_source['status'] = 'sale'
car_source['source_type'] = 'cpersonal'
car_source['process_status'] = 'S'
car_source['global_sibling'] = 0
car_source['city_sibling'] = 0
car_source['dealer_sibling'] = 0
car_source['domain_sibling'] = 0
car_source['expired_at'] = datetime.datetime.now() + datetime.timedelta(days=10)
car_source['domain_sibling'] = 0
car_source['created_on'] = cur_time 
car_source['cs_clean'] = 0
car_source = car_source.merge(pcm, how='left', on=['city'])
car_source = car_source.merge(car_autohome_all, how='left', on=['detail_slug'])
car_source = car_source.merge(gpj_detail,how='left',on=['detail_slug','volume','control'])
car_source = car_source.rename(columns={'origin_name':'title','detail_model_slug':'model_detail_slug','popular':'popularity'})
car_source = car_source.drop(['create_time','detail_slug'],axis=1)
car_source = car_source.loc[(car_source['model_detail_slug'].notnull()),:].reset_index(drop=True)
car_source = car_source.drop_duplicates(['url'])
car_source['url'] = car_source.apply(process_url, axis=1)
car_source['eval_price'] = car_source.apply(process_eval_pirce, axis=1)
car_source[['gpj_index','rank']] = car_source.apply(process_gpj_index, axis=1)
car_source[['thumbnail','car_image']] = car_source.apply(process_thumbnail, axis=1)
car_source = car_source.loc[(car_source['car_image'].notnull()),:].reset_index(drop=True)
car_source = car_source.sort_values(by=['url','pub_time'],ascending=[True,False]).reset_index(drop=True)
car_source = car_source.drop_duplicates(['url']).reset_index(drop=True)
# car_source.drop(['source_id','car_image','description','conditions','contact','region','company_name',
#                 'maintenance','insurance_money','quality_assurance','transfer_owner','annual_insurance',
#                 'compulsory_insurance','business_insurance','price_bn'],axis=1).to_csv('../tmp/train/clean_car_source.csv',index=False)
car_source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 48 columns):
source_id               89 non-null object
url                     89 non-null object
title                   89 non-null object
year                    89 non-null float64
month                   89 non-null float64
mile                    89 non-null float64
city                    89 non-null object
color                   0 non-null float64
phone                   89 non-null float64
thumbnail               89 non-null object
price                   89 non-null float64
domain                  89 non-null object
transfer_owner          89 non-null float64
annual_insurance        89 non-null object
compulsory_insurance    89 non-null object
business_insurance      89 non-null object
description             89 non-null object
conditions              89 non-null object
contact                 89 non-null object
region                  89 non-null object
company_name            89 non-

In [66]:
import re
import pandas as pd
from sqlalchemy import create_engine

# car_source = pd.read_csv('./clean_car_source.csv',low_memory=False)

ENCODING = 'utf-8'
PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
PRODUCE_DB_USER = 'pingjia'
PRODUCE_DB_PASSWD = 'De32wsxC'
PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'

engine = create_engine(PRODUCE_PINGJIA_ENGINE, encoding=ENCODING)  

with engine.begin() as con:
    columns = str(list(car_source.columns))
    columns = columns[1:len(columns) - 1]
    columns = columns.replace('\'', '')
    for i in range(i, len(car_source)):
        url = car_source.loc[i, 'url']
        value = str(list(car_source.loc[i, :].values))
        value = value[1:len(value) - 1]
        value = re.sub(r' nan', ' null', value)
        sql = 'INSERT INTO car_source(' + columns + ') VALUES (' + value + ') ON DUPLICATE KEY UPDATE url=\'' +url+ '\' '
        con.execute(sql)
        
        sql = 'select id,thumbnail from car_source where url=\'' +url+ '\' '
        query_car_source = pd.read_sql_query(sql, engine)
        final_car_source = car_source.loc[i:i,:].merge(query_car_source,how='left',on=['thumbnail'])
        car_image = pd.DataFrame()
        car_image['image'] = pd.Series(final_car_source.loc[0,'car_image'])
        car_id = final_car_source.loc[0,'id']
        car_image['car_id'] = car_id
        car_image['create_time'] = cur_time
        sql = 'delete from car_image where car_id = ' + str(car_id)
        con.execute(sql)
        car_image.to_sql(name='car_image', if_exists='append', con=engine, index=False)
        
        car_detail_info = final_car_source.copy()
        car_detail_info = car_detail_info.loc[:,['conditions','description','contact','region','company_name','annual_insurance','compulsory_insurance',
                                                'business_insurance','transfer_owner','maintenance','insurance_money','quality_assurance','id','price_bn']]
        car_detail_info['update_time'] = cur_time.strftime('%Y-%m-%d %H:%M:%S')
        car_detail_info['condition_score'] = ''
        car_detail_info = car_detail_info.rename(columns={'id':'car_id','annual_insurance':'examine_insurance','compulsory_insurance':'mandatory_insurance',
                                                         'conditions':'condition'})
        car_detail_info['mdn_status'] = 'P'
        car_detail_info['dealer_status'] = 0
        sql = 'delete from car_detail_info where car_id = ' + str(car_id)
        con.execute(sql)
        car_detail_info.to_sql(name='car_detail_info', if_exists='append', con=engine, index=False)
con.close()


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '101.201.148.49' (timed out)") (Background on this error at: http://sqlalche.me/e/e3q8)

### 三.入库car_image

In [86]:
import pandas as pd
from sqlalchemy import create_engine

# 测试库
ENCODING = 'utf-8'
# PRODUCE_DB_ADDR_OUTTER = '101.201.148.49'
PRODUCE_DB_ADDR_OUTTER = '101.201.143.74'
PRODUCE_DB_USER = 'leidengjun'
PRODUCE_DB_PASSWD = 'ldj_DEV_~!'
PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'

engine = create_engine(PRODUCE_PINGJIA_ENGINE)
url = 'www.guazi.com/anshan/3007396545x.htm'
with engine.begin() as con:   
    sql = 'select id,thumbnail from car_source where url=\'' +url+ '\' '
    query_car_source = pd.read_sql_query(sql, engine)
    final_car_source = car_source.loc[0:0,:].merge(query_car_source,how='left',on=['thumbnail'])
    car_image = pd.DataFrame()
    car_image['image'] = pd.Series(final_car_source.loc[0,'car_image'])
    car_id = final_car_source.loc[0,'id']
    car_image['car_id'] = car_id
    car_image['create_time'] = cur_time
    sql = 'delete from car_image where car_id = ' + str(car_id)
    con.execute(sql)
    car_image.to_sql(name='car_image', if_exists='append', con=engine, index=False)

    car_detail_info = final_car_source.copy()
    car_detail_info = car_detail_info.loc[:,['conditions','description','contact','region','company_name','annual_insurance','compulsory_insurance',
                                            'business_insurance','transfer_owner','maintenance','insurance_money','quality_assurance','id','price_bn']]
    car_detail_info['update_time'] = cur_time.strftime('%Y-%m-%d %H:%M:%S')
    car_detail_info['condition_score'] = ''
    car_detail_info = car_detail_info.rename(columns={'id':'car_id','annual_insurance':'examine_insurance','compulsory_insurance':'mandatory_insurance',
                                                     'conditions':'condition'})
    car_detail_info['mdn_status'] = 'P'
    car_detail_info['dealer_status'] = 0
    sql = 'delete from car_detail_info where car_id = ' + str(car_id)
    con.execute(sql)
    car_detail_info.to_sql(name='car_detail_info', if_exists='append', con=engine, index=False)
con.close()

In [87]:
car_detail_info

,condition,description,contact,region,company_name,examine_insurance,mandatory_insurance,business_insurance,transfer_owner,maintenance,insurance_money,quality_assurance,car_id,price_bn,update_time,condition_score,mdn_status,dealer_status
0,经检测，发动机变数箱运转正常，工况完好，各电器设备无异常。局部钣金。,这辆车已经在路上行驶五年多了。经常开着跑来跑去的，所以公里数跑的稍多。我的车基本都没怎么出过...,赵先生,千山,瓜子二手车,2019.11,2019.11,2019.11,0.0,NaN,NaN,NaN,129574714,14.99,2019-03-05 16:00:32,,P,0


In [50]:
final_car_source

'2019-03-05 15:09:32'

In [ ]:
final_car_source = car_source.merge(query_car_source.loc[:,['id','thumbnail']],how='left',on=['thumbnail'])

if len(final_car_source) == len(car_source):
    print('数据匹配正常!')
else:
    print('数据匹配异常,请检查!')
    
car_image = pd.DataFrame()
for i in range(0,len(car_source)):
    temp = pd.DataFrame()
    temp['image'] = pd.Series(final_car_source.loc[i,'car_image'])
    temp['car_id'] = final_car_source.loc[i,'id']
    car_image = car_image.append(temp,sort=False).reset_index(drop=True)

car_image['create_time'] = cur_time
car_image.to_csv('../tmp/train/car_image_origin.csv',index=False)
car_image.info()

### 四.入库car_detail_info

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# 测试库
ENCODING = 'utf-8'
# PRODUCE_DB_ADDR_OUTTER = '101.201.148.49'
PRODUCE_DB_ADDR_OUTTER = '101.201.143.74'
PRODUCE_DB_USER = 'leidengjun'
PRODUCE_DB_PASSWD = 'ldj_DEV_~!'
PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'

# query_sql = 'select id,model_detail_slug,thumbnail from pingjia.car_source where created_on = \'' + cur_time.strftime('%Y-%m-%d %H:%M:%S') + '\' '
query_sql = 'select id,model_detail_slug from pingjia.car_source where created_on > \'2019-03-01 00:00:00\' '

engine = create_engine(PRODUCE_PINGJIA_ENGINE)
query_car_source = pd.read_sql_query(query_sql, engine)

In [ ]:
car_detail_info = query_car_source.copy()
car_detail_info['update_time'] = cur_time.strftime('%Y-%m-%d %H:%M:%S')
car_detail_info['condition'] = ''
car_detail_info['condition_score'] = ''
car_detail_info['company_name'] = '瓜子二手车'
car_detail_info = car_detail_info.rename(columns={'id':'car_id'})
car_detail_info['mdn_status'] = 'P'
car_detail_info['dealer_status'] = 0
car_detail_info.drop(['model_detail_slug'],axis=1).to_csv('../tmp/train/car_detail_info.csv',index=False)
car_detail_info.info()